In [ ]:
!pip install oaklib==0.5.16-rc2 llm

In [2]:
#from oaklib.selector import get_adapter
#hpsemsim = get_adapter(f"semsimian:sqlite:obo:hp")

In [ ]:
!runoak -i semsimian:sqlite:obo:hp similarity -p i,p .all @ .all -o hp_all_vs_all.tsv

[00:00:54] Building all X all pairwise similarity: ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 2%A[00:00:00] Building all X all pairwise similarity: ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 0%